In [1]:
!pwd
import sys
%reload_ext autoreload
%autoreload 2
%matplotlib inline
sys.executable

In [2]:
# tensorflow2.0 + kerasでGPUメモリの使用量を抑える方法(最小限だけ使うように設定)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [3]:
import sys
sys.path.append(r'C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLP_for_ensemble\20201124')
import moa_MLPs_funcs

Tensorflow version 2.3.1
n_classes: 206
DEBUG


In [6]:
import optuna


def objective(trial):
    model_type = "stacked_tabnet"
    
    params = dict(
        epsilon=1e-05,
        feature_columns=None,  # データセットのTensorflow特徴列
        virtual_batch_size=None,  # 仮想バッチサイズ。全体のバッチサイズは virtual_batch_size の整数倍じゃないとだめらしい
        num_layers=2,  # 重ねるTabNetsの数
        num_decision_steps=1,  # decision stepsの数
        batch_momentum=trial.suggest_categorical("batch_momentum", [0.9, 0.95, 0.98]),  # 仮想バッチのMomentum。よくわからん
        norm_type=trial.suggest_categorical("norm_type", ["group", "batch"]),  # 正規化のタイプ
        relaxation_factor=trial.suggest_categorical("gamma", [1.0, 1.2, 1.5, 2.0]),  # 論文の探索範囲
        num_groups=trial.suggest_categorical("num_groups", [-1, 1, 2, 3, 4, 5]),  # group normarizaionのグループの数。よくわからん
        sparsity_coefficient=trial.suggest_categorical("lambda_sparse", [0, 0.000001, 0.0001, 0.001, 0.01, 0.1]),  # 論文の探索範囲。sparsity正則化
    )
    # feature_dim must be larger than output dim
    # feature_dim must be a list of length `num_layers`
    params["feature_dim"] = trial.suggest_categorical("Na", [8, 16, 24, 32, 64, 128])  # feature transformation block
    params["output_dim"] = 4  # params["feature_dim"]  # int(trial.suggest_uniform("Nd", params["feature_dim"] // 8, params["feature_dim"]))  # decision step

    print("-" * 100)
    print(f"params: {params}")

    oof_score, Y_pred = moa_MLPs_funcs.train_and_evaluate(
        model_type=model_type, params=params
    )

    return oof_score

In [7]:
%%time

n_trials = 100
#n_trials = 50
#n_trials = 3

study = optuna.create_study(
    study_name="study",
    storage=f"sqlite:///study.db",
    load_if_exists=True,
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=1),
)
study.optimize(objective, n_trials=n_trials)
study.trials_dataframe().to_csv(f"objective_history.csv", index=False)
with open(f"objective_best_params.txt", mode="w") as f:
    f.write(str(study.best_params))
print(f"\nstudy.best_params:\n{study.best_params}")

[I 2020-11-25 22:12:15,571] A new study created in memory with name: study
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


----------------------------------------------------------------------------------------------------
params: {'epsilon': 1e-05, 'feature_columns': None, 'virtual_batch_size': None, 'num_layers': 2, 'num_decision_steps': 1, 'batch_momentum': 0.95, 'norm_type': 'batch', 'relaxation_factor': 1.0, 'num_groups': -1, 'sparsity_coefficient': 1e-06, 'feature_dim': 32, 'output_dim': 4}
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epoch 1/2
12/12 [==============================] - ETA: 0s - loss: 0.6954WARNING:tensorflow:Layer stacked_tab_net_classifier is casting 

c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epoch 1/2
11/12 [==========================>...] - ETA: 0s - loss: 0.6944WARNING:tensorflow:Layer stacked_tab_net_classifier is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all

[I 2020-11-25 22:13:18,482] Trial 0 finished with value: 0.6388073541867332 and parameters: {'batch_momentum': 0.95, 'norm_type': 'batch', 'gamma': 1.0, 'num_groups': -1, 'lambda_sparse': 1e-06, 'Na': 32}. Best is trial 0 with value: 0.6388073541867332.


...

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epoch 1/2
11/12 [==========================>...] - ETA: 0s - loss: 0.6951WARNING:tensorflow:Layer stacked_tab_net_classifier is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all

c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epoch 1/2
11/12 [==========================>...] - ETA: 0s - loss: 0.6947WARNING:tensorflow:Layer stacked_tab_net_classifier is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all

[I 2020-11-25 22:15:32,604] Trial 2 finished with value: 0.634337633751469 and parameters: {'batch_momentum': 0.95, 'norm_type': 'group', 'gamma': 1.5, 'num_groups': 1, 'lambda_sparse': 0.0001, 'Na': 64}. Best is trial 2 with value: 0.634337633751469.


oof_score: 0.634337633751469
----------------------------------------------------------------------------------------------------

study.best_params:
{'batch_momentum': 0.95, 'norm_type': 'group', 'gamma': 1.5, 'num_groups': 1, 'lambda_sparse': 0.0001, 'Na': 64}
Wall time: 3min 17s
